In [2]:
## 필요라이브러리
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import folium

In [38]:
# 모든 정거장 위치 검색하기
allBusStop = []
def busStop ():
    url = 'http://apis.data.go.kr/6260000/BusanBIMS/busStopList'
    params ={'serviceKey' : 'l2SER5UdFnVMTCfW/75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ==',
              'pageNo' : '1',
              'numOfRows' : '499'}

    response = requests.get(url, params=params)
    decodecontent = response.content.decode('utf-8') 
    xmlParser = BeautifulSoup(decodecontent,'xml')
    items= xmlParser.select("item")
    for number in range(0,len(items)):
        item = items[number].text.split('\n')
        item.remove('')
        item.remove('')
        allBusStop.append(item)
busStop()

In [ ]:
busnStopDF = pd.read_excel('./data/부산광역 시_부산버스정보시스템.xlsx')

In [ ]:
condition = busnStopDF['정류장 구분'] == '일반'
stoptGeneral = busnStopDF[condition]
stoptGeneral.to_excel('./data/부산버스_일반.xlsx',index=False)

In [ ]:
condition = busnStopDF['정류장 구분'] == '마을'
stopMura = busnStopDF[condition]

In [ ]:
stopMura.to_excel('./data/부산버스_마을.xlsx',index=False)

In [4]:
stop_general = pd.read_excel('./data/부산버스_일반.xlsx')
location = stop_general[['GPS X 좌표','GPS Y 좌표']]

In [6]:
map =folium.Map(location=[location['GPS Y 좌표'].mean(),location['GPS X 좌표'].mean()],zoom_start=12)


for i in stop_general.index:
    lat = stop_general.loc[i,'GPS Y 좌표']
    lng = stop_general.loc[i,'GPS X 좌표']
    stop_name = stop_general.loc[i,'정류소 명']
    folium.CircleMarker(location=[lat,lng],
                  fill=True,fill_color='green',fill_opacity=2,color='blue',radius=2,weight=1,popup=stop_name).add_to(map)
map.save('./data/부산버스 정류소 위치.html')